# Single Values (with statistical features)

* by Firuz Juraev (Combined Master/PhD student)

In [1]:
import pandas as pd 
from datetime import datetime
import seaborn as sns
import glob
from os import listdir 
import os.path
from os import path
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [2]:
def side_by_side(*objs, **kwds):
    from pandas.io.formats.printing import adjoin
    space = kwds.get('space', 4)
    reprs = [repr(obj).split('\n') for obj in objs]
    print (adjoin(space, *reprs))
    print()
    return

### Variables 

In [3]:
directory = "Data_3145_Stat_24/" 

### Functions 

In [4]:
def list_files(directory):
    files = []
    counter = 0
    for f in listdir(directory):
        if f.endswith('.' + "csv"):
            files.append(f)
            counter = counter + 1
    print ("CSV Files: " + str(counter))
    return files

In [5]:
list_files(directory)

CSV Files: 8


['datetimeevents_24.csv',
 'inputevents_24.csv',
 'microbiologyevents_24.csv',
 'neonates_admissions.csv',
 'neonates_icustays.csv',
 'neonates_patients.csv',
 'outputevents_24.csv',
 'prescriptions_24.csv']

### Load unique neonates 

In [89]:
neonates_ids = (pd.read_csv("FinalData/SingleValue/birth_weight.csv"))['HADM_ID'].to_frame()

In [90]:
neonates_ids.shape

(3135, 1)

### Admissions 

In [86]:
admissions = pd.read_csv(directory + "neonates_admissions.csv")

In [87]:
admissions.head()

,subject_id,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_LOCATION,DISCHARGE_LOCATION,RELIGION,ETHNICITY,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,NaN,PHYS REFERRAL/NORMAL DELI,HOME,CATHOLIC,WHITE,0,1
1,39,106266,2114-11-29 21:04:00,2114-12-09 15:10:00,NaN,PHYS REFERRAL/NORMAL DELI,HOME,NOT SPECIFIED,UNKNOWN/NOT SPECIFIED,0,1
2,358,110872,2168-10-24 23:48:00,2168-10-29 03:23:00,NaN,PHYS REFERRAL/NORMAL DELI,HOME,CATHOLIC,WHITE,0,1
3,363,196503,2176-03-01 15:26:00,2176-03-03 14:04:00,NaN,CLINIC REFERRAL/PREMATURE,HOME,PROTESTANT QUAKER,BLACK/AFRICAN AMERICAN,0,1
4,50,132761,2112-06-23 19:40:00,2112-06-26 10:15:00,NaN,PHYS REFERRAL/NORMAL DELI,HOME,OTHER,WHITE,0,1


In [91]:
admissions = pd.merge(admissions, neonates_ids, on='HADM_ID', how='inner') 

neonates_ids = admissions['subject_id'].to_frame()

neonates_ids.shape

(3135, 1)

In [21]:
neonates_ids.rename(columns={'subject_id': 'SUBJECT_ID'}, inplace=True)

## Detecting where neonate passed away [Done] - 1

### Patients 

In [8]:
patients = pd.read_csv(directory + "PATIENTS.csv")

In [22]:
patients = pd.merge(patients, neonates_ids, on='SUBJECT_ID', how='inner') 

In [9]:
patients.head(2)

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1


In [23]:
patients.shape 

(3135, 8)

In [24]:
patients[patients.DOD_SSN.isnull() == False]

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
66,488,517,F,2158-09-09 00:00:00,2159-01-04 00:00:00,2159-01-04 00:00:00,2159-01-04 00:00:00,1
2954,15002,15870,F,2170-07-23 00:00:00,2172-02-28 00:00:00,NaN,2172-02-28 00:00:00,1


In [36]:
dead_neonates = patients[patients.EXPIRE_FLAG == 1]['SUBJECT_ID'].to_frame()

In [25]:
admissions.shape 

(3135, 11)

In [29]:
admissions[admissions['HOSPITAL_EXPIRE_FLAG'] == 1].head() 

,subject_id,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_LOCATION,DISCHARGE_LOCATION,RELIGION,ETHNICITY,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
45,513,103472,2186-12-15 20:40:00,2186-12-25 13:00:00,2186-12-25 13:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,HISPANIC OR LATINO,1,1
46,517,155972,2158-09-09 15:48:00,2159-01-04 20:09:00,2159-01-04 20:09:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,OTHER,BLACK/AFRICAN AMERICAN,1,1
51,607,155563,2158-12-16 23:08:00,2158-12-17 08:50:00,2158-12-17 08:50:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,OTHER,WHITE,1,1
129,1558,142939,2181-07-18 18:55:00,2181-07-19 18:56:00,2181-07-19 18:56:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,WHITE,1,1
151,1334,138015,2137-09-02 14:43:00,2137-09-02 12:00:00,2137-09-02 12:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,PROTESTANT QUAKER,WHITE,1,1


In [32]:
admissions[admissions['HOSPITAL_EXPIRE_FLAG'] == 1].shape 

(48, 11)

In [40]:
dead_neonates['subject_id'].nunique()

50

In [35]:
admissions[admissions['subject_id'] == 15870] 

,subject_id,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_LOCATION,DISCHARGE_LOCATION,RELIGION,ETHNICITY,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
1542,15870,125678,2170-07-23 22:17:00,2170-09-21 17:00:00,NaN,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,NOT SPECIFIED,HISPANIC OR LATINO,0,1


In [37]:
dead_neonates.rename(columns={'SUBJECT_ID':'subject_id'}, inplace=True)

In [41]:
dead_neonates = pd.merge(admissions, dead_neonates, on='subject_id', how='inner')

dead_neonates

,subject_id,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_LOCATION,DISCHARGE_LOCATION,RELIGION,ETHNICITY,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,513,103472,2186-12-15 20:40:00,2186-12-25 13:00:00,2186-12-25 13:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,HISPANIC OR LATINO,1,1
1,517,155972,2158-09-09 15:48:00,2159-01-04 20:09:00,2159-01-04 20:09:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,OTHER,BLACK/AFRICAN AMERICAN,1,1
2,607,155563,2158-12-16 23:08:00,2158-12-17 08:50:00,2158-12-17 08:50:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,OTHER,WHITE,1,1
3,1558,142939,2181-07-18 18:55:00,2181-07-19 18:56:00,2181-07-19 18:56:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,WHITE,1,1
4,1334,138015,2137-09-02 14:43:00,2137-09-02 12:00:00,2137-09-02 12:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,PROTESTANT QUAKER,WHITE,1,1
5,1587,111817,2179-03-04 19:50:00,2179-03-06 19:29:00,2179-03-06 19:29:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,CATHOLIC,WHITE,1,1
6,2181,192593,2137-12-16 21:32:00,2137-12-18 11:30:00,2137-12-18 11:30:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,OTHER,1,1
7,2532,169421,2103-02-25 04:34:00,2103-02-28 05:10:00,2103-02-28 05:10:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,UNOBTAINABLE,UNKNOWN/NOT SPECIFIED,1,1
8,3071,160514,2158-11-08 20:29:00,2158-11-16 21:12:00,2158-11-16 21:12:00,PHYS REFERRAL/NORMAL DELI,DEAD/EXPIRED,UNOBTAINABLE,WHITE,1,1
9,3576,176428,2190-12-23 11:11:00,2190-12-29 23:41:00,2190-12-29 23:41:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,NaN,BLACK/AFRICAN AMERICAN,1,1


In [42]:
dead_neonates.drop(['RELIGION', 'HAS_CHARTEVENTS_DATA'], axis=1, inplace=True)

In [43]:
dead_neonates.head()

,subject_id,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_LOCATION,DISCHARGE_LOCATION,ETHNICITY,HOSPITAL_EXPIRE_FLAG
0,513,103472,2186-12-15 20:40:00,2186-12-25 13:00:00,2186-12-25 13:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,HISPANIC OR LATINO,1
1,517,155972,2158-09-09 15:48:00,2159-01-04 20:09:00,2159-01-04 20:09:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,BLACK/AFRICAN AMERICAN,1
2,607,155563,2158-12-16 23:08:00,2158-12-17 08:50:00,2158-12-17 08:50:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,WHITE,1
3,1558,142939,2181-07-18 18:55:00,2181-07-19 18:56:00,2181-07-19 18:56:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,WHITE,1
4,1334,138015,2137-09-02 14:43:00,2137-09-02 12:00:00,2137-09-02 12:00:00,CLINIC REFERRAL/PREMATURE,DEAD/EXPIRED,WHITE,1


## Labeling Death Location - 2 

* neonate [15870] - passed away at home 
* neonate [18093] - passed away at short term hospital 

In [ ]:
patients = pd.read_csv(directory + "PATIENTS.csv")

## Extracting 3135 neonates 
patients = pd.merge(patients, neonates_ids, on='SUBJECT_ID', how='inner') 

In [44]:
patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,242,258,F,2124-09-19 00:00:00,NaN,NaN,NaN,0
1,243,260,F,2105-03-23 00:00:00,NaN,NaN,NaN,0
2,646,685,F,2126-07-29 00:00:00,NaN,NaN,NaN,0
3,647,686,F,2193-10-05 00:00:00,NaN,NaN,NaN,0
4,652,692,F,2118-11-29 00:00:00,NaN,NaN,NaN,0


In [45]:
patients.shape 

(3135, 8)

In [47]:
patients.drop(['DOD', 'DOD_HOSP', 'DOD_SSN', 'ROW_ID'], axis=1, inplace=True)

In [48]:
patients.head()

,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG
0,258,F,2124-09-19 00:00:00,0
1,260,F,2105-03-23 00:00:00,0
2,685,F,2126-07-29 00:00:00,0
3,686,F,2193-10-05 00:00:00,0
4,692,F,2118-11-29 00:00:00,0


In [50]:
# POD = place of Death
patients["POD"] = 0 

In [60]:
counter = 0 
# Labeling patients that passed away at NICU 
while counter < len(patients):
    if patients['EXPIRE_FLAG'].iloc[counter] == 1: 
        patients.loc[counter, 'POD'] = 1
    
    # Patient passed away at hospital 
    if patients['SUBJECT_ID'].iloc[counter] == 18093: 
        patients.loc[counter, 'POD'] = 2 
        
    # Patient passed away at home  
    if patients['SUBJECT_ID'].iloc[counter] == 15870: 
        patients.loc[counter, 'POD'] = 3  
        
    counter = counter + 1 

In [62]:
patients[patients.EXPIRE_FLAG == 1]

,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,POD
65,513,F,2186-12-15 00:00:00,1,1
66,517,F,2158-09-09 00:00:00,1,1
74,607,F,2158-12-16 00:00:00,1,1
88,1334,M,2137-09-02 00:00:00,1,1
118,2181,M,2137-12-16 00:00:00,1,1
145,2532,M,2103-02-25 00:00:00,1,1
149,3576,F,2190-12-23 00:00:00,1,1
235,1558,M,2181-07-18 00:00:00,1,1
236,1587,M,2179-03-04 00:00:00,1,1
325,4412,F,2141-05-03 00:00:00,1,1


In [92]:
admissions.columns

Index(['subject_id', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME',
       'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'RELIGION', 'ETHNICITY',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'],
      dtype='object')

In [93]:
admissions.drop(['subject_id', 'ADMITTIME', 'DEATHTIME',
       'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'RELIGION', 'ETHNICITY',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA'], axis=1, inplace=True)

In [94]:
admissions.head(2)

,HADM_ID,DISCHTIME
0,106266,2114-12-09 15:10:00
1,110872,2168-10-29 03:23:00


In [72]:
admissions.rename(columns={'subject_id':'SUBJECT_ID'}, inplace=True)

In [73]:
patients = pd.merge(patients, admissions, on='SUBJECT_ID', how='inner') 

patients.head(2)

,SUBJECT_ID,GENDER,DOB,EXPIRE_FLAG,POD,HADM_ID,ADMITTIME,DISCHARGE_LOCATION,ETHNICITY
0,258,F,2124-09-19 00:00:00,0,0,189406,2124-09-19 03:59:00,HOME,ASIAN
1,260,F,2105-03-23 00:00:00,0,0,190363,2105-03-23 10:23:00,HOME,WHITE


I come back here because I deleted DISCHTIME. So that I did some processes again 

In [95]:
patients = pd.merge(patients, admissions, on='HADM_ID', how='inner') 

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS,DISCHTIME
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,HOME,0,0,0.0,2124-09-22 15:52:00
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,HOME,0,0,0.0,2105-03-30 11:00:00


I am changing order of columns 

In [96]:
patients = patients[['SUBJECT_ID','HADM_ID','DOB','GENDER', 'ETHNICITY', 'ADMITTIME', 'DISCHTIME', 'DISCHARGE_LOCATION',
                     'EXPIRE_FLAG', 'POD', 'DATEEVENTS']]

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,0.0
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,0.0


In [74]:
patients.shape 

(3135, 9)

## Statistical Features - 3 

### 3.1 DateTimeEvents

In [64]:
dateevents = pd.read_csv(directory + "datetimeevents_24.csv") 

dateevents.head(2)

,HADM_ID,subject_id,CHARTTIME,ADMITTIME
0,106266,39,2114-11-29 22:00:00,2114-11-29 21:04:00
1,106266,39,2114-11-29 23:00:00,2114-11-29 21:04:00


In [65]:
dateevents.rename(columns={"subject_id": "SUBJECT_ID"}, inplace=True)

In [66]:
dateevents.drop(['CHARTTIME'], axis=1, inplace=True)

In [81]:
dateevents['HADM_ID'].nunique() 

1380

In [67]:
(dateevents.groupby(['HADM_ID']).size()).to_csv("count.csv")

dateevents_count = pd.read_csv("count.csv")

dateevents_count.rename(columns={'0': "COUNT"}, inplace=True)

dateevents_count.head(2)

,HADM_ID,COUNT
0,100044,10
1,100062,3


In [75]:
dateevents_count.shape 

(1380, 2)

#### Merge with patients  

In [76]:
patients.columns

Index(['SUBJECT_ID', 'GENDER', 'DOB', 'EXPIRE_FLAG', 'POD', 'HADM_ID',
       'ADMITTIME', 'DISCHARGE_LOCATION', 'ETHNICITY'],
      dtype='object')

#### Changing order of patients 

In [77]:
patients = patients[['SUBJECT_ID','HADM_ID','DOB','GENDER', 'ETHNICITY', 'ADMITTIME', 'DISCHARGE_LOCATION',
                     'EXPIRE_FLAG', 'POD']]

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,HOME,0,0
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,HOME,0,0


In [78]:
patients = pd.merge(patients, dateevents_count, on='HADM_ID', how='left') 

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,COUNT
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,HOME,0,0,NaN
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,HOME,0,0,NaN


In [84]:
patients.rename(columns={'COUNT': 'DATEEVENTS'}, inplace=True)

patients.isnull().sum()

SUBJECT_ID               0
HADM_ID                  0
DOB                      0
GENDER                   0
ETHNICITY                0
ADMITTIME                0
DISCHARGE_LOCATION       0
EXPIRE_FLAG              0
POD                      0
DATEEVENTS            1759
dtype: int64

In [85]:
patients["DATEEVENTS"].fillna(0, inplace = True)

patients.isnull().sum()

SUBJECT_ID            0
HADM_ID               0
DOB                   0
GENDER                0
ETHNICITY             0
ADMITTIME             0
DISCHARGE_LOCATION    0
EXPIRE_FLAG           0
POD                   0
DATEEVENTS            0
dtype: int64

### 3.2 Inputevents 

In [97]:
inputevents = pd.read_csv(directory + "inputevents_24.csv") 

inputevents.head(2)

,HADM_ID,subject_id,CHARTTIME,ITEMID,AMOUNT,LABEL,ADMITTIME
0,106266,39,2114-11-29 21:30:00,30187,NaN,D10W,2114-11-29 21:04:00
1,106266,39,2114-11-29 22:00:00,30187,2.9,D10W,2114-11-29 21:04:00


In [99]:
inputevents['HADM_ID'].nunique()

3091

In [100]:
D10W = inputevents[inputevents.LABEL == 'D10W']
D10W.head(2)

,HADM_ID,subject_id,CHARTTIME,ITEMID,AMOUNT,LABEL,ADMITTIME
0,106266,39,2114-11-29 21:30:00,30187,NaN,D10W,2114-11-29 21:04:00
1,106266,39,2114-11-29 22:00:00,30187,2.9,D10W,2114-11-29 21:04:00


In [101]:
D10W.groupby(['HADM_ID']).size().to_frame().to_csv("count.csv")
inputevents_D10W_count = pd.read_csv("count.csv")
inputevents_D10W_count.rename(columns={'0':'COUNT'}, inplace=True)
inputevents_D10W_count.head(2)

,HADM_ID,COUNT
0,100029,12
1,100044,25


In [102]:
patients = pd.merge(patients, inputevents_D10W_count, on='HADM_ID', how='left') 

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS,COUNT
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,0.0,22.0
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,0.0,23.0


In [103]:
patients.rename(columns={'COUNT': 'DW10_COUNT'}, inplace=True)

patients.isnull().sum()

SUBJECT_ID              0
HADM_ID                 0
DOB                     0
GENDER                  0
ETHNICITY               0
ADMITTIME               0
DISCHTIME               0
DISCHARGE_LOCATION      0
EXPIRE_FLAG             0
POD                     0
DATEEVENTS              0
DW10_COUNT            438
dtype: int64

In [104]:
patients["DW10_COUNT"].fillna(0, inplace = True)

patients.isnull().sum()

SUBJECT_ID            0
HADM_ID               0
DOB                   0
GENDER                0
ETHNICITY             0
ADMITTIME             0
DISCHTIME             0
DISCHARGE_LOCATION    0
EXPIRE_FLAG           0
POD                   0
DATEEVENTS            0
DW10_COUNT            0
dtype: int64

### Calculating Average Amount of Input  

In [105]:
def fillNaN(df, filename): 
    if pd.isnull(df['AMOUNT'].iloc[0]) and df['row'].iloc[0] == 1: 
        df['AMOUNT'].fillna(method='bfill', inplace=True)
    else: 
        df['AMOUNT'].fillna(method='ffill', inplace=True)
    
    if path.exists(filename): 
        df.to_csv(filename, mode='a', header=False, index=False)
    else: 
        df.to_csv(filename, index=False, header=True)

In [106]:
## Sorting Data 
D10W.sort_values(by=['HADM_ID', 'CHARTTIME'], inplace=True) 

## Giving row id by group 
D10W['row'] = D10W.groupby(['HADM_ID']).cumcount()+1

## Finding only unique values 
unique_hadm_ids = D10W['HADM_ID'].unique()

## Filename 
filename = 'inputevents_D10W.csv'

for i in range(0, len(unique_hadm_ids)): 
    fillNaN(D10W.loc[D10W['HADM_ID'] == unique_hadm_ids[i]], directory + filename)
    
## Checking 
inputevents_D10W = pd.read_csv(directory + filename)

side_by_side(inputevents_D10W.isnull().sum(), inputevents_D10W.count())

HADM_ID        0    HADM_ID       58642
subject_id     0    subject_id    58642
CHARTTIME      0    CHARTTIME     58642
ITEMID         0    ITEMID        58642
AMOUNT        34    AMOUNT        58608
LABEL          0    LABEL         58642
ADMITTIME      0    ADMITTIME     58642
row            0    row           58642
dtype: int64        dtype: int64       



In [107]:
inputevents_D10W.dropna(inplace=True)

side_by_side(inputevents_D10W.isnull().sum(), inputevents_D10W.count())

HADM_ID       0    HADM_ID       58608
subject_id    0    subject_id    58608
CHARTTIME     0    CHARTTIME     58608
ITEMID        0    ITEMID        58608
AMOUNT        0    AMOUNT        58608
LABEL         0    LABEL         58608
ADMITTIME     0    ADMITTIME     58608
row           0    row           58608
dtype: int64       dtype: int64       



In [108]:
inputevents_D10W.columns

Index(['HADM_ID', 'subject_id', 'CHARTTIME', 'ITEMID', 'AMOUNT', 'LABEL',
       'ADMITTIME', 'row'],
      dtype='object')

In [110]:
inputevents_D10W.drop(['CHARTTIME', 'LABEL', 'ADMITTIME', 'subject_id', 
       'row', 'ITEMID'], axis=1, inplace=True)

inputevents_D10W.head(2)

,HADM_ID,AMOUNT
0,100029,4.4
1,100029,5.7


In [111]:
inputevents_D10W.groupby(['HADM_ID']).sum().to_csv("sum.csv")
d10w_sum = pd.read_csv("sum.csv")
d10w_sum.rename(columns={'0':'D10W_SUM'}, inplace=True)
d10w_sum.head(2)

,HADM_ID,AMOUNT
0,100029,66.5
1,100044,168.8


In [114]:
d10w_sum.shape 

(2695, 2)

In [112]:
inputevents_D10W.groupby(['HADM_ID']).mean().to_csv("mean.csv")
d10w_mean = pd.read_csv("mean.csv")
d10w_mean.rename(columns={'0':'D10W_MEAN'}, inplace=True)
d10w_mean.head(2)

,HADM_ID,AMOUNT
0,100029,5.541667
1,100044,6.752000


In [113]:
d10w_mean.shape 

(2695, 2)

In [116]:
d10w_sum.rename(columns={'AMOUNT':'D10W_SUM'}, inplace=True)

d10w_mean.rename(columns={'AMOUNT':'D10W_MEAN'}, inplace=True)

In [117]:
d10w_mean.head(2)

,HADM_ID,D10W_MEAN
0,100029,5.541667
1,100044,6.752000


In [118]:
patients = pd.merge(patients, d10w_sum, on='HADM_ID', how='left')
patients = pd.merge(patients, d10w_mean, on='HADM_ID', how='left')

In [119]:
patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS,DW10_COUNT,D10W_SUM,D10W_MEAN
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,0.0,22.0,116.400001,5.290909
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,0.0,23.0,233.400002,10.147826


In [120]:
patients.fillna(0, inplace=True)

In [121]:
patients.isnull().sum()

SUBJECT_ID            0
HADM_ID               0
DOB                   0
GENDER                0
ETHNICITY             0
ADMITTIME             0
DISCHTIME             0
DISCHARGE_LOCATION    0
EXPIRE_FLAG           0
POD                   0
DATEEVENTS            0
DW10_COUNT            0
D10W_SUM              0
D10W_MEAN             0
dtype: int64

###  3.3 Output Events 

In [122]:
outputevents = pd.read_csv(directory + "outputevents_24.csv") 

outputevents.head(2)

,HADM_ID,subject_id,CHARTTIME,ITEMID,VALUE,CGID,LABEL,ADMITTIME
0,106266,39,2114-11-30 05:00:00,43175,9.0,15658,Urine .,2114-11-29 21:04:00
1,106266,39,2114-11-30 13:00:00,43175,33.0,14440,Urine .,2114-11-29 21:04:00


In [124]:
outputevents.drop(['ITEMID', 'CGID', 'subject_id', 'ADMITTIME'], axis=1, inplace=True)

In [125]:
outputevents.head(2)

,HADM_ID,CHARTTIME,VALUE,LABEL
0,106266,2114-11-30 05:00:00,9.0,Urine .
1,106266,2114-11-30 13:00:00,33.0,Urine .


In [126]:
outputevents_Urine = outputevents[outputevents['LABEL'] == 'Urine .']

In [127]:
outputevents_Urine['HADM_ID'].nunique()

2794

### Filling Data 

In [128]:
outputevents_Urine.isnull().sum()

HADM_ID        0
CHARTTIME      0
VALUE        326
LABEL          0
dtype: int64

In [129]:
def fillNaN(df, filename): 
    if pd.isnull(df['VALUE'].iloc[0]) and df['row'].iloc[0] == 1: 
        df['VALUE'].fillna(method='bfill', inplace=True)
    else: 
        df['VALUE'].fillna(method='ffill', inplace=True)
    
    if path.exists(filename): 
        df.to_csv(filename, mode='a', header=False, index=False)
    else: 
        df.to_csv(filename, index=False, header=True)

In [130]:
## Sorting Data 
outputevents_Urine.sort_values(by=['HADM_ID', 'CHARTTIME'], inplace=True) 

## Giving row id by group 
outputevents_Urine['row'] = outputevents_Urine.groupby(['HADM_ID']).cumcount()+1

## Finding only unique values 
unique_hadm_ids = outputevents_Urine['HADM_ID'].unique()

## Filename 
filename = 'outputevents_Urine.csv'

for i in range(0, len(unique_hadm_ids)): 
    fillNaN(outputevents_Urine.loc[outputevents_Urine['HADM_ID'] == unique_hadm_ids[i]], filename)
    
## Checking 
check_df = pd.read_csv(filename)

side_by_side(check_df.isnull().sum(), check_df.count())

HADM_ID       0    HADM_ID      12342
CHARTTIME     0    CHARTTIME    12342
VALUE        11    VALUE        12331
LABEL         0    LABEL        12342
row           0    row          12342
dtype: int64       dtype: int64      



In [131]:
check_df.dropna(inplace=True)

In [132]:
check_df.head(2)

,HADM_ID,CHARTTIME,VALUE,LABEL,row
0,100029,2185-04-17 12:00:00,0.0,Urine .,1
1,100044,2195-08-12 20:00:00,16.0,Urine .,1


In [133]:
check_df.groupby(['HADM_ID']).size().to_frame().to_csv("count.csv")
urine_count = pd.read_csv("count.csv")
urine_count.rename(columns={'0':'URINE_COUNT'}, inplace=True)
urine_count.head(2)

,HADM_ID,URINE_COUNT
0,100029,1
1,100044,4


In [134]:
check_df.groupby(['HADM_ID']).sum().to_csv("sum.csv")
urine_sum = pd.read_csv("sum.csv")
urine_sum.rename(columns={'0':'URINE_SUM'}, inplace=True)
urine_sum.head(2)

,HADM_ID,VALUE,row
0,100029,0.0,1
1,100044,108.0,10


In [135]:
urine_sum.rename(columns={'VALUE':'URINE_SUM'}, inplace=True)
urine_sum.drop(['row'], axis=1, inplace=True)

In [136]:
urine_sum.head(2)

,HADM_ID,URINE_SUM
0,100029,0.0
1,100044,108.0


In [137]:
check_df.groupby(['HADM_ID']).mean().to_csv("mean.csv")
urine_mean = pd.read_csv("mean.csv")
urine_mean.rename(columns={'VALUE':'URINE_AVG'}, inplace=True)
urine_mean.drop(['row'], axis=1, inplace=True)
urine_mean.head(2)

,HADM_ID,URINE_AVG
0,100029,0.0
1,100044,27.0


In [138]:
patients = pd.merge(patients, urine_count, on='HADM_ID', how='left')
patients = pd.merge(patients, urine_sum, on='HADM_ID', how='left')
patients = pd.merge(patients, urine_mean, on='HADM_ID', how='left')

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS,DW10_COUNT,D10W_SUM,D10W_MEAN,URINE_COUNT,URINE_SUM,URINE_AVG
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,0.0,22.0,116.400001,5.290909,5.0,108.0,21.600000
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,0.0,23.0,233.400002,10.147826,6.0,122.0,20.333333


In [140]:
side_by_side(patients.isnull().sum(), patients.count())

SUBJECT_ID              0    SUBJECT_ID            3135
HADM_ID                 0    HADM_ID               3135
DOB                     0    DOB                   3135
GENDER                  0    GENDER                3135
ETHNICITY               0    ETHNICITY             3135
ADMITTIME               0    ADMITTIME             3135
DISCHTIME               0    DISCHTIME             3135
DISCHARGE_LOCATION      0    DISCHARGE_LOCATION    3135
EXPIRE_FLAG             0    EXPIRE_FLAG           3135
POD                     0    POD                   3135
DATEEVENTS              0    DATEEVENTS            3135
DW10_COUNT              0    DW10_COUNT            3135
D10W_SUM                0    D10W_SUM              3135
D10W_MEAN               0    D10W_MEAN             3135
URINE_COUNT           358    URINE_COUNT           2777
URINE_SUM             358    URINE_SUM             2777
URINE_AVG             358    URINE_AVG             2777
dtype: int64                 dtype: int64       

In [141]:
patients.fillna(0, inplace = True)

### 3.4 Neonates Microbiologyevents

In [142]:
microbiologyevents = pd.read_csv(directory + "microbiologyevents_24.csv") 

microbiologyevents.head(2)

,HADM_ID,subject_id,CHARTTIME,SPEC_ITEMID,ORG_ITEMID,AB_ITEMID,ADMITTIME
0,106266,39,2114-11-29 21:45:00,70014.0,0.0,0.0,2114-11-29 21:04:00
1,104518,66,2188-08-25 00:45:00,70014.0,0.0,0.0,2188-08-25 00:14:00


In [143]:
microbiologyevents['HADM_ID'].nunique()

2989

In [144]:
microbiologyevents.groupby(['HADM_ID']).size().to_frame().to_csv("count.csv")
microbiologyevents_count = pd.read_csv("count.csv")
microbiologyevents_count.rename(columns={'0':'MICROBIOLOGY_TEST'}, inplace=True)
microbiologyevents_count.head(2)

,HADM_ID,MICROBIOLOGY_TEST
0,100029,1
1,100044,1


In [146]:
org_items = microbiologyevents[microbiologyevents['ORG_ITEMID'] > 0]

org_items.head(2)

,HADM_ID,subject_id,CHARTTIME,SPEC_ITEMID,ORG_ITEMID,AB_ITEMID,ADMITTIME
52,155563,607,2158-12-17 00:20:00,70014.0,80002.0,90026.0,2158-12-16 23:08:00
53,155563,607,2158-12-17 00:20:00,70014.0,80002.0,90029.0,2158-12-16 23:08:00


In [147]:
org_items.groupby(['HADM_ID']).size().to_frame().to_csv("count.csv")
org_count = pd.read_csv("count.csv")
org_count.rename(columns={'0':'NEGATIVE_RESULT'}, inplace=True)
org_count.head(2)

,HADM_ID,NEGATIVE_RESULT
0,100545,4
1,101232,4


In [148]:
org_count.shape 

(23, 2)

In [149]:
patients = pd.merge(patients, microbiologyevents_count, on='HADM_ID', how='left')
patients = pd.merge(patients, org_count, on='HADM_ID', how='left')

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,DATEEVENTS,DW10_COUNT,D10W_SUM,D10W_MEAN,URINE_COUNT,URINE_SUM,URINE_AVG,MICROBIOLOGY_TEST,NEGATIVE_RESULT
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,0.0,22.0,116.400001,5.290909,5.0,108.0,21.600000,1.0,NaN
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,0.0,23.0,233.400002,10.147826,6.0,122.0,20.333333,1.0,NaN


In [150]:
patients.fillna(0, inplace = True)

side_by_side(patients.isnull().sum(), patients.count())

SUBJECT_ID            0    SUBJECT_ID            3135
HADM_ID               0    HADM_ID               3135
DOB                   0    DOB                   3135
GENDER                0    GENDER                3135
ETHNICITY             0    ETHNICITY             3135
ADMITTIME             0    ADMITTIME             3135
DISCHTIME             0    DISCHTIME             3135
DISCHARGE_LOCATION    0    DISCHARGE_LOCATION    3135
EXPIRE_FLAG           0    EXPIRE_FLAG           3135
POD                   0    POD                   3135
DATEEVENTS            0    DATEEVENTS            3135
DW10_COUNT            0    DW10_COUNT            3135
D10W_SUM              0    D10W_SUM              3135
D10W_MEAN             0    D10W_MEAN             3135
URINE_COUNT           0    URINE_COUNT           3135
URINE_SUM             0    URINE_SUM             3135
URINE_AVG             0    URINE_AVG             3135
MICROBIOLOGY_TEST     0    MICROBIOLOGY_TEST     3135
NEGATIVE_RESULT       0    N

### 3.5 Neonates Prescriptions

In [151]:
prescriptions = pd.read_csv(directory + 'prescriptions_24.csv')

prescriptions.head(2)

,HADM_ID,subject_id,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DOSE_VAL_RX,ADMITTIME
0,104518,66,2188-08-25 00:00:00,2188-08-27 00:00:00,BASE,Syringe (Neonatal) *D5W*,2.4,2188-08-25 00:14:00
1,104518,66,2188-08-25 00:00:00,2188-08-27 00:00:00,MAIN,NEO*IV*AMPicillin Sodium,470,2188-08-25 00:14:00


In [152]:
prescriptions.groupby(['HADM_ID']).size().to_frame().to_csv("count.csv")
prescriptions_count = pd.read_csv("count.csv")
prescriptions_count.rename(columns={'0':'PRESCRIPTIONS'}, inplace=True)
prescriptions_count.head(2)

,HADM_ID,PRESCRIPTIONS
0,100029,4
1,100044,9


In [153]:
prescriptions_count.shape 

(2163, 2)

In [154]:
patients = pd.merge(patients, prescriptions_count, on='HADM_ID', how='left')

patients.fillna(0, inplace=True)

side_by_side(patients.isnull().sum(), patients.count())

SUBJECT_ID            0    SUBJECT_ID            3135
HADM_ID               0    HADM_ID               3135
DOB                   0    DOB                   3135
GENDER                0    GENDER                3135
ETHNICITY             0    ETHNICITY             3135
ADMITTIME             0    ADMITTIME             3135
DISCHTIME             0    DISCHTIME             3135
DISCHARGE_LOCATION    0    DISCHARGE_LOCATION    3135
EXPIRE_FLAG           0    EXPIRE_FLAG           3135
POD                   0    POD                   3135
DATEEVENTS            0    DATEEVENTS            3135
DW10_COUNT            0    DW10_COUNT            3135
D10W_SUM              0    D10W_SUM              3135
D10W_MEAN             0    D10W_MEAN             3135
URINE_COUNT           0    URINE_COUNT           3135
URINE_SUM             0    URINE_SUM             3135
URINE_AVG             0    URINE_AVG             3135
MICROBIOLOGY_TEST     0    MICROBIOLOGY_TEST     3135
NEGATIVE_RESULT       0    N

## LENGTH OF STAY 

In [156]:
start = datetime.now()
patients['DISCHTIME'] = patients['DISCHTIME'].astype('datetime64[ns]')
patients['ADMITTIME'] = patients['ADMITTIME'].astype('datetime64[ns]')
print("Computing time takes %s" %(datetime.now()-start))

Computing time takes 0:00:00.011511


In [158]:
patients['LOS'] = (patients['DISCHTIME'] - patients['ADMITTIME']) / np.timedelta64(1, 's')

patients['LOS'] = patients['LOS'] / 24 / 60 / 60

patients.head(2)

,SUBJECT_ID,HADM_ID,DOB,GENDER,ETHNICITY,ADMITTIME,DISCHTIME,DISCHARGE_LOCATION,EXPIRE_FLAG,POD,...,DW10_COUNT,D10W_SUM,D10W_MEAN,URINE_COUNT,URINE_SUM,URINE_AVG,MICROBIOLOGY_TEST,NEGATIVE_RESULT,PRESCRIPTIONS,LOS
0,258,189406,2124-09-19 00:00:00,F,ASIAN,2124-09-19 03:59:00,2124-09-22 15:52:00,HOME,0,0,...,22.0,116.400001,5.290909,5.0,108.0,21.600000,1.0,0.0,0.0,3.495139
1,260,190363,2105-03-23 00:00:00,F,WHITE,2105-03-23 10:23:00,2105-03-30 11:00:00,HOME,0,0,...,23.0,233.400002,10.147826,6.0,122.0,20.333333,1.0,0.0,4.0,7.025694


### Saving into file 

In [159]:
directory_SV = "FinalData/SingleValue/"

In [161]:
patients.to_csv(directory_SV + "patients_single_value_data.csv", index=False)

THE END